# **Trust Region Policy Optimization**

Environment: Lunar Lander

## References

#### Papers
- [Trust Region Policy Optimization, Schulman et al. 2015](https://arxiv.org/abs/1502.05477)
- [High-Dimensional Continuous Control Using Generalized Advantage Estimation, Schulman et al, 2015. Algorithm: GAE.](https://arxiv.org/abs/1506.02438)
- [Approximately Optimal Approximate Reinforcement Learning, Kakade and Langford 2002](https://people.eecs.berkeley.edu/~pabbeel/cs287-fa09/readings/KakadeLangford-icml2002.pdf)

#### Blogs
- [OpenAI Spinning Up - Trust Region Policy Optimization](https://spinningup.openai.com/en/latest/algorithms/trpo.html)

#### Others
- [OpenAI Spinning Up](https://spinningup.openai.com/en/latest/index.html)
- [OpenAI Gym](https://gym.openai.com/)

## Preparation

In [ ]:
%%capture
!sudo apt update
!sudo apt install python-opengl xvfb -y
!pip install gym[box2d] pyvirtualdisplay piglet tqdm

%%capture
from pyvirtualdisplay import Display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

%matplotlib inline
import matplotlib.pyplot as plt

from IPython import display

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
from tqdm import tqdm_notebook

In [ ]:
# Import gym and create a Lunar Lander environment
# Observation/State: Box(8,)
# Action: Discrete(4)
%%capture
import gym
env = gym.make('LunarLander-v2')

## TRPO Algorithm
Trust Region Policy Optimization algorithm from the [original paper](https://arxiv.org/abs/1502.05477)

> Initialize $\pi_0$.
> <br>**for** i = 0, 1, 2,... until convergence **do**

> > Compute all advantage values $A_{\pi_i}(s, a)$. 
> > <br> Solve the constrained optimization problem:
> > <br>
> > <br> $\pi_{i+1} = \underset{\pi}{arg\ max}\ L_{\pi_i}(\pi)$ $~~~~$ s.t. $\bar{D}^{\rho_{\pi_i}}_{KL}(\pi_i, \pi) \leq \delta$
> > <br> where $L_{\pi_i}(\pi) = \eta(\pi_i) + \underset{s}{\sum} \rho_{\pi_i}(s) \underset{a}{\sum} \pi(a|s) A_{\pi_i}(s, a)$

> > which is equivalent to solve:
> > <br> $\underset{\theta}{maximize}\ \underset{s}{\sum} \rho_{\theta_{old}}(s) \underset{a}{\sum} \pi_\theta(a|s) A_{\theta_{old}}(s, a)$ $~~~~$ s.t. $\bar{D}^{\rho_{\theta_{old}}}_{KL}(\theta_{old}, \theta) \leq \delta$

> **end for**

<br> Next, we are trying to approximate the objective and constraint functions using Monte Carlo simulation.

1. Replace $\underset{s}{\sum} \rho_{\theta_{old}}(s)[...]$ by $\frac{1}{1-\gamma}\mathbb{E}_{s \sim \rho_{\theta_{old}}}[...]$.
2. Replace advantage values function $A_{\theta_{old}}$ by the state-action value function $Q_{\theta_{old}}$.
3. Replace sum over actions by importance sampling estimator ($q$ is the sampling distribution):
> $\underset{a}{\sum} \pi_\theta(a|s) A_{\theta_{old}}(s_n, a) = \mathbb{E}_{a \sim q}\ [\frac{\pi_{\theta}(a|s_n)}{q(a|s_n)} A_{\theta_{old}}(s_n, a)]$

<br> The optimization problem become:

> $\underset{\theta}{maximize}\ \mathbb{E}_{s \sim \rho_{\theta_{old}},\ a \sim q}\ [\frac{\pi_{\theta}(a|s)}{q(a|s)} Q_{\theta_{old}}(s, a)]$ $~~~~$ s.t. $\mathbb{E}_{s \sim \rho_{\theta_{old}}}\ [\ D_{KL}(\ \pi_{\theta_{old}}(\cdot|s)\ \|\ \pi_{\theta}(\cdot|s)\ )\ ] \leq \delta$

<br> The remaining steps are:

1. Replace the expectations by sample averages.
2. Replace the $Q$ value by an empirical estimate.

In the original paper, the author provided two schemes, single path and vine.

* `Single Path`
<br> In this scheme, we generate a trajectory ($s_0, a_0, ... , s_{T-1}, a_{T-1}, s_T$) by $\rho_0$ and $\pi_{old}$, which means $p(a|s) = \pi_{old}(a|s)$, and compute the $Q$ value as $$\hat{\mathcal{Q}}_{\theta_{old}}(s_t, a_t) = \sum_{\tau\in\theta_{old}} \sum_{l=0}^T \gamma^lr(s_{t+l})$$

* `Vine`
<br> In this scheem, 

### Approximations
The theoretical TRPO,

<br> $\underset{\theta}{maximize}\ \mathbb{E}_{s \sim \rho_{\theta_{old}},\ a \sim q}\ [\frac{\pi_{\theta}(a|s)}{q(a|s)} Q_{\theta_{old}}(s, a)]$ $~~~~$ s.t. $\mathbb{E}_{s \sim \rho_{\theta_{old}}}\ [\ D_{KL}(\ \pi_{\theta_{old}}(\cdot|s)\ \|\ \pi_{\theta}(\cdot|s)\ )\ ] \leq \delta$,

<br> is still not easy to work with. Therefore, in Appendix C of the [original paper](https://arxiv.org/abs/1502.05477), the authors make additional approximations to solve the optimization problem efficiently.

1. `Compute a search direction`
> Compute a search direction, using a linear approximation to objective and quagratic approximation to the constraint.

2. `Line search`
> Perform a line search in that direction, ensuring that we improve the nonlinear objective while satisfying the nonlinear constraint.

### Pseudocode
In the [original paper](https://arxiv.org/abs/1502.05477), the advantage estimates $\hat{A}_t$ is $Q$ value $\hat{\mathcal{Q}}_{\theta_{old}}(s_t, a_t)$.
<br> <br>

1. Input: initail policy parameters $\theta_0$, initial value function parameters $\phi_0$.

2. Hyperparameters: KL-divergence limit $\delta$, backtracking coefficient $\alpha$, maximum number of backtracking steps $K$.

3. **for** $k=0, 1, 2, ...$ **do**

4. > Collect set of trajectories $D_k = {\{\tau_i\}}$ by running policy $\pi_k = \pi(\theta_k)$ in the environment. (Single Path or Vine) 

5. > Compute rewards-to-go $\hat{R}_t$.

6. > Compute advantage estimates, $\hat{A}_t$ (using any method of advantage estimation) based on the current value function $V_{\phi_k}$.

7. > Estimate policy gradient as 

<br> 
    $$\hat{g}_k = \frac{1}{|D_k|} \sum_{\tau \in D_k} \sum_{t=0}^T \ \nabla_\theta\ log\ \pi_\theta(a_t|s_t)|_{\theta_k}\ \hat{A}_t.$$
<br>

8. > Use the conjugate gradient algorithm to compute 

<br> 
    $$\hat{x}_k$$ 
<br>

### Policy Gradient Network

> In this notebook, we implement two policy gradient networks, the one used as an example in hw15 of HY-Lee's course (PGNetLee) and the one in the [original paper](https://arxiv.org/abs/1502.05477) (PGNet).

In [ ]:
class PGNetLee(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 4)

    def forward(self, state):
        hid = torch.tanh(self.fc1(state))
        hid = torch.tanh(self.fc2(hid))
        return F.softmax(self.fc3(hid), dim=-1)


#class PGNet(nn.Module):

#    def __init__(self):
#       super().__init__()

### Trust Region Policy Optimization Agent

> Next, we build an TRPO agent which take the policy gradient network abouve to take action and have the following functions:
1. `learn()`: Update the policy network with log probabilities and rewards.
2. `sample()`: Sample an action and return the log probabilities by the policy network with the observation from the environment.

In [ ]:
class TRPOAgent():

    def __init__(self, network, gamma=0.9):
        self.network = network
        self.optimizer = optim.SGD(self.network.parameters(), lr=0.001)
        self.gamma = gamma  # Discount parameter $\phi$

    def learn(self, log_probs, advantages):
        loss = 0   # Surrogate advantage
        
        # Compute search direction (Using conjugate gradient algorithm)
        

        # Update policy network by backpropagating line search

        # (Option) Fit value function

    def sample(self, state):
        # Compute the probability of each action with the observation from the environment
        action_prob = self.network(torch.FloatTensor(state))
        action_dist = Categorical(action_prob)

        # Sample an action
        action = action_dist.sample()

        # The probability of the sampled action
        log_prob = action_dist.log_prob(action)

        return action.item(), log_prob

### Estimation: State-Action Value Function

$$\mathcal{Q}_{\theta_{old}}(s_t, a_t) = \sum_{l=0}^T \gamma^lr(s_{t+l})$$


In [ ]:
def computeQ(episode_rewards, gamma):
    Qvalues = episode_rewards.copy()
    gamma = gamma
    for i in reversed(range(len(Qvalues))):
            if (i+1) < len(Qvalues):
                Qvalues[i] = Qvalues[i] + gamma*Qvalues[i+1]
    return Qvalues

### Train

#### Single Path

In [ ]:
# Initialize policy network $\theta_0$
network = PolicyGradientNetwork()

# Initialize policy agent $\gamma$
agent = TRPOAgent(network=network)

# Hyperparameters
delta = 0       # KL-divergence limit $\delta$
alpha = 0       # Backpropagation coefficient $\alpha$
NUM_BATCH = 0   # Maximum number of backpropagation steps $K$

In [ ]:
# Turn the neural network to training mode
agent.network.train()

# Hyperparameters
EPISODE_PER_BATCH = 5  # Update agent once per EPISODE_PER_BATCH episodes.

# Log parameters
avg_total_rewards, avg_final_rewards = [], []

# Start training
prg_bar = tqdm_notebook(range(NUM_BATCH))
for batch in prg_bar:

    log_probs = []
    total_rewards, final_rewards, batch_Qvalues = [], [], [], []

    # Collect training data
    for episode in range(EPISODE_PER_BATCH):
        
        state = env.reset()
        total_reward, total_step = 0, 0

        episode_rewards, episode_Qvalues = [], []

        while True:

            action, log_prob = agent.sample(state)
            next_state, reward, done, _ = env.step(action)

            log_probs.append(log_prob)
            state = next_state
            total_reward += reward
            total_step += 1

            episode_rewards.append(reward)

            if done:
                final_rewards.append(reward)
                total_rewards.append(total_reward)
                episode_Qvalues = computeQ(episode_rewards, agent.gamma)

                batch_Qvalues.append(episode_Qvalues)
                break

    # Log training process
    avg_total_reward = sum(total_rewards) / len(total_rewards)
    avg_final_reward = sum(final_rewards) / len(final_rewards)
    avg_total_rewards.append(avg_total_reward)
    avg_final_rewards.append(avg_final_reward)
    prg_bar.set_description(f"Total: {avg_total_reward: 4.1f}, Final: {avg_final_reward: 4.1f}")

    # Q values normalization and standardization
    batch_Qvalues = np.concatenate(batch_Qvalues, axis=0)
    batch_Qvalues = (batch_Qvalues - np.mean(batch_Qvalues)) / np.std(batch_Qvalues) + 1e-9
    # Update Policy Gradient Network
    agent.learn(torch.stack(log_probs), torch.from_numpy(batch_Qvalues))

#### Vine

In [ ]:
network = PolicyGradientNetwork()
agent = TRPOAgent(network)

### Test